# PETs/TETs – Hyperledger Aries – Manufacturer 1 (Holder)

In [ ]:
%%javascript
document.title ='🏙️ City Agent'

## PART 2: Issue a VC to the Manufacturer Agents

**What:** -

**Why:** -

**How:** <br>


**Accompanying Notebooks:** `02_issue_VC_manufacturers.ipynb`

### 0 - Setup
#### 0.1 - Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController
import libs.helpers as helpers
from libs.agent_connection_manager import CredentialHolder
import os
from termcolor import colored

#### 0.2 – Variables

In [ ]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

In [6]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [7]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

#### 1.3 – Init ACM issuing authority

In [ ]:
# The CredentialHolder registers relevant webhook servers and event listeners
city_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC (if no, there is no need to execute the notebook)
city_agent.get_credentials()


## 2 – Establish a connection with the Authority agent
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the City requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the city's notebook.

### 2.1 Create invitation to Authority agent

In [ ]:
# Setup for connection with Authority agent
alias = None
auto_accept = True # Accept response of Authority agent right away
public = False # Do not use public DID
multi_use = False # Invitation is only for one invitee
label = label

connection_id = city_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, label=label)

<div style="font-size: 25px"><center><b>Break Point 2</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🏛</center></div><br>
<center>Please switch to the Authority agent 🏛: Execute Step 2.1, and enter the invitation when prompted.</center>

---

## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [11]:
city_agent.send_message(connection_id, "testing connection")

Copy the following dict and past it into the Authority agency
{
    '@id': '1a4ef9a6-2dc4-4eae-b6b2-6003b1903aa2',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['2HBfxB6MbCXv8tDAriinYY5BzT8gZUtUYKAQ1VXDNDmF'],
    'serviceEndpoint': 'https://80a67f24c0cb.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request


In [ ]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [13]:
basic_message = '{"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}'
await agent_controller.messaging.send_message(connection_id, basic_message)

{'thread_id': '7079dec1-8d30-4ec1-9431-edfc11f8f6e9'}

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  7a04ff63-a269-44a5-94af-decb75ba51f7
State :  active
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
Connection ID: 7a04ff63-a269-44a5-94af-decb75ba51f7 is now active.


**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [14]:
identifiers = fnc.get_identifiers()
schema_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
schema_id

{}

/tmp/ipykernel_21/3503404827.py:75: RuntimeWarning: coroutine 'MessagingController.send_message' was never awaited
  agent_controller.messaging.send_message(connection_id, "This is a response from Bob")


Handle message 7a04ff63-a269-44a5-94af-decb75ba51f7
{
    'connection_id': '7a04ff63-a269-44a5-94af-decb75ba51f7',
    'content': 'Hello City. Please provide me your city and country.',
    'message_id': '948fb0ff-e570-4eba-a9e4-ae007749db41',
    'sent_time': '2021-08-09 14:19:22.306175Z',
    'state': 'received',
}


In [ ]:
city_agent.request_vc(connection_id, schema_id)

### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [15]:
city_agent.get_credentials()

{}


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : 7a04ff63-a269-44a5-94af-decb75ba51f7
Credential exchange ID : cbc6367d-3e65-4705-8c7a-e24c5af0d96a
Agent Protocol Role :  holder
Protocol State :  offer_received

---------------------------------------------------

Handle Credential Webhook Payload
Credential Offer Recieved
The proposal dictionary is likely how you would understand and display a credential offer in your application

 {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '6341c9c8-6c12-4bdf-858e-59169269f2e9', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'city', 'value': 'Berlin'}, {'name': 'country', 'value': 'DE'}, {'name': 'isCityAgency', 'value': 'TRUE'}]}, 'schema_id': 'MTXSc4YD8wixyM9ekZbmMC:2:certify-city-agency:0.0.1', 'comment': 'Issuing VC that City is an agency', 'cred_def_id':

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [19]:
await agent_controller.terminate()